In [48]:
# DigitRecognizer LeNet5 Full MNIST
#
# CNN: LetNet5 v2
#
# Kaggle Scoring: 0.99900, Position #30 (Top 2%) (21/1/2018)
#
# Seed: 1
# Epochs: 200
# Batch Size: 200
# Train metrics: {'recall': 1.0, 'loss': 1.3700266e-05, 'global_step': 114951, 'f1_score': 1.0, 'accuracy': 1.0, 'precision': 1.0}
# Val metrics: {'recall': 0.99955761, 'loss': 0.040703569, 'global_step': 114951, 'f1_score': 0.99966816871350095, 'accuracy': 0.99360001, 'precision': 0.99977875}
# Test metrics: {'recall': 0.99955654, 'loss': 0.032167114, 'global_step': 114951, 'f1_score': 0.99955654144287098, 'accuracy': 0.99409997, 'precision': 0.99955654}    

In [ ]:
import numpy as np
import tensorflow as tf

In [83]:
SEED = 1

BATCH_SIZE = 200
EPOCHS = 200

def onehot_to_dense( onehot):
    locations = np.where(onehot)[1]

    return np.reshape(locations, (-1,1))

In [25]:
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
mnist = read_data_sets("MNIST_data/", one_hot=True)

# 55,000 data points of training data (mnist.train)
# 5,000 points of validation data (mnist.validation)
# 10,000 points of test data (mnist.test)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [49]:
# 70k total samples
print("train shape: {}".format(mnist.train._images.shape))    #(55000, 784)   91%
print("val shape: {}".format(mnist.validation._images.shape)) #(5000, 784)     9%
print("test shape: {}".format(mnist.test._images.shape))      #(10000, 784)   14%

train shape: (55000, 784)
val shape: (5000, 784)
test shape: (10000, 784)


In [84]:
#label,pixel0,pixel1,pixel2,..,pixel783
X_train = np.reshape( mnist.train._images, (-1,28,28,1))
X_val   = np.reshape( mnist.validation._images, (-1,28,28,1))
X_test  = np.reshape( mnist.test._images, (-1,28,28,1))

y_train = onehot_to_dense( mnist.train._labels)
y_val   = onehot_to_dense( mnist.validation._labels)
y_test  = onehot_to_dense( mnist.test._labels)

print("X_train.shape: {}".format(X_train.shape))
print("X_val.shape: {}".format(X_val.shape))
print("X_test.shape: {}".format(X_test.shape))

print("y_train.shape: {}".format(y_train.shape))
print("y_val.shape: {}".format(y_val.shape))

X_train.shape: (55000, 28, 28, 1)
X_val.shape: (5000, 28, 28, 1)
X_test.shape: (10000, 28, 28, 1)
y_train.shape: (55000, 1)
y_val.shape: (5000, 1)


In [70]:
# Model: Lenet-5 v3

def cnn_model_fn( features, labels, mode):
    # Conv1: 14x14@30
    model = tf.layers.conv2d( 
        inputs = features["x"], 
        filters = 30, 
        kernel_size = [5, 5],
        strides = 1, 
        padding = "SAME",
        activation = tf.nn.relu)
    model = tf.layers.max_pooling2d( 
        inputs = model,
        pool_size = [ 2, 2],
        strides = 2)
    
    # Conv2: 7x7@16
    model = tf.layers.conv2d( 
        inputs = model, 
        filters = 16, 
        kernel_size = [3, 3],
        strides = 1, 
        padding = "SAME",
        activation = tf.nn.relu)
    model = tf.layers.max_pooling2d( 
        inputs = model,
        pool_size = [ 2, 2],
        strides = 2)

    model = tf.layers.dropout( model, rate=0.2, training=(mode == tf.estimator.ModeKeys.TRAIN))
    
    model = tf.layers.flatten( inputs = model) # 784 nodes

    # FC
    model = tf.layers.dense( 
        inputs = model,
        units = 120,
        activation = tf.nn.relu)

    model = tf.layers.dropout( model, rate=0.2, training=(mode == tf.estimator.ModeKeys.TRAIN))
        
    model = tf.layers.dense( 
        inputs = model,
        units = 84,
        activation = tf.nn.relu)

    # Output (softmax)
    logits = tf.layers.dense( model, 10, name = "output_tensor")
    
    predictions = {
        "classes": tf.argmax( logits, axis=1),
        "probabilities": tf.nn.softmax( logits, name="softmax_tensor")
    }
    
    
    # PREDICT
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode = mode, predictions = predictions)
    
    
    # TRAIN
    onehot_labels = tf.one_hot( indices = tf.cast(labels, tf.int32), depth = 10)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits( 
        logits=logits, 
        labels=onehot_labels), name="cost_tensor")
    
    if mode == tf.estimator.ModeKeys.TRAIN:
        train_op = tf.train.AdamOptimizer().minimize( cost, global_step=tf.train.get_global_step())
        return tf.estimator.EstimatorSpec(mode = mode, loss = cost, train_op = train_op)
    
    
    # EVAL
    eval_metrics_ops = {
        "accuracy": tf.metrics.accuracy(
            labels = labels, 
            predictions = predictions["classes"]),
        
        "precision": tf.metrics.precision(
            labels = labels,
            predictions = predictions["classes"]),
        
        "recall": tf.metrics.recall(
            labels = labels,
            predictions = predictions["classes"])
    }
    return tf.estimator.EstimatorSpec( mode = mode, loss = cost, eval_metric_ops = eval_metrics_ops)

In [71]:
def train_model( X_train, y_train):
    model = tf.estimator.Estimator(
        model_fn = cnn_model_fn,
        model_dir="/tmp/cnn_lenet5_v1_digits_model")

    model.train(
        input_fn = tf.estimator.inputs.numpy_input_fn(
            x = {"x": X_train},
            y = y_train,
            batch_size = BATCH_SIZE,
            num_epochs = EPOCHS,
            shuffle = True))
    
    return model

In [76]:
def evaluate_model( model, features, labels):

    def f1_score( precision, recall):
        return 2.0 / (precision**-1 + recall**-1)
    
    # Model performance
    metrics = model.evaluate(
        input_fn = tf.estimator.inputs.numpy_input_fn(
            x = {"x": features},
            y = labels,
            batch_size = BATCH_SIZE,
            num_epochs = 1,
            shuffle = False))

    metrics["f1_score"] = f1_score( metrics["precision"], metrics["recall"])

    return metrics

In [81]:
def kaggle_make_prediction( model, test_filename, output_filename):
    data_test  = np.genfromtxt( test_filename,  delimiter=',', dtype=np.float32)
    X_test  = data_test[1:,:]              #(28000, 784)
    X_test  = X_test.reshape((-1,28,28,1)) #(42000, 28, 28, 1)
    X_test  = X_test / 255.0

    predictions = model.predict(
        input_fn = tf.estimator.inputs.numpy_input_fn(
            x = {"x": X_test},
            num_epochs = 1,
            shuffle = False))

    csv = open( output_filename, "w")
    csv.write("ImageId,Label\n")
    for i, pred in enumerate(predictions):
        csv.write("{},{}\n".format( i+1, pred["classes"]))
    csv.close()

In [85]:
tf.logging.set_verbosity(tf.logging.ERROR)

with tf.Session() as sess:
    tf.set_random_seed( SEED)
    model = train_model( X_train, y_train)
    
    print(">> Train metrics: %r"% evaluate_model( model, X_train, y_train))
    print(">> Val metrics: %r"% evaluate_model( model, X_val, y_val))
    
    print(">> Test metrics: %r"% evaluate_model( model, X_test, y_test))
    
    kaggle_make_prediction( model, "test.csv", "digit_prediction.csv")

>> Train metrics: {'recall': 1.0, 'loss': 1.3700266e-05, 'global_step': 114951, 'f1_score': 1.0, 'accuracy': 1.0, 'precision': 1.0}
>> Val metrics: {'recall': 0.99955761, 'loss': 0.040703569, 'global_step': 114951, 'f1_score': 0.99966816871350095, 'accuracy': 0.99360001, 'precision': 0.99977875}
>> Test metrics: {'recall': 0.99955654, 'loss': 0.032167114, 'global_step': 114951, 'f1_score': 0.99955654144287098, 'accuracy': 0.99409997, 'precision': 0.99955654}
